In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
src=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(src, 'html.parser')
jnk=soup.prettify()

In [3]:
table_post=soup.find('table',class_='wikitable sortable')
table_rows=table_post.find_all('tr')
#each tr tag within the table of postal codes have been retrieved as a list, from here continue to ,make the dataframe later using the rules mentioned in the coursera page

In [4]:
df_toronto = pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])


In [5]:
postal_codes=[]
for i in table_rows[1:]:
    postal_codes.append(i.td.text)
postal_codes=list(set(postal_codes)) #getting the unique postal codes   

In [6]:
dic_post={}
for i in postal_codes:
    list_places=['']
    dic_post[i]=list_places


In [7]:
for i in table_rows[1:]:
    row=i.find_all('td')
    temp=dic_post[row[0].text]
    temp[0]=row[1].text
    emp_list=[]
    temp.append(emp_list)
    temp=temp[0:2]
    dic_post[row[0].text]=temp
    


In [8]:
for i in table_rows[1:]:
    row=i.find_all('td')
    new_list=dic_post[row[0].text]
    temp=new_list[1]
    temp.append(row[2].text.strip('\n'))
    new_list[1]=temp
    dic_post[row[0].text]=new_list


In [9]:
df_toronto = pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])
#Generating the pd dataframe
#Also assigning Borough name as the neighborhood names to the places with neighborhood name given as not assigned
for i in dic_post:
    tmp_list=dic_post[i]
    neighb_list=tmp_list[1]
    neighb_list = [w.replace('Not assigned', tmp_list[0]) for w in neighb_list]  #Assigning the borough name as the neighborhood name for neighborhoods with not assigned
    df_toronto=df_toronto.append({'Postal Code' : i , 'Borough' : tmp_list[0],'Neighborhood':neighb_list} , ignore_index=True)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M2R,North York,[Willowdale West]
1,M2B,Not assigned,[Not assigned]
2,M9Z,Not assigned,[Not assigned]
3,M6Z,Not assigned,[Not assigned]
4,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"


In [10]:
df_toronto=df_toronto[df_toronto['Borough']!='Not assigned'] #removing all the postal codes that are not assigned
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M2R,North York,[Willowdale West]
4,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
5,M4B,East York,"[Woodbine Gardens, Parkview Hill]"
6,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
8,M4P,Central Toronto,[Davisville North]


In [11]:
neighb_tmp=df_toronto['Neighborhood'] #converting the neighborhood column from a list to neighborhood names split by ,
i=0
fin_neighb_tmp=[]
for i in neighb_tmp:
    tmp=i
    fin_neighb_tmp.append(','.join(tmp))
df_toronto['Neighborhood']=fin_neighb_tmp

In [12]:
#final dataframe after wrangling
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M2R,North York,Willowdale West
4,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
5,M4B,East York,"Woodbine Gardens,Parkview Hill"
6,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
8,M4P,Central Toronto,Davisville North


In [27]:
#Now let's move onto getting the latitudes and longitudes of each borough in Toronto
df_lat=pd.read_csv('https://cocl.us/Geospatial_data')
df_lat.head()
df_lat_ind=df_lat.set_index('Postal Code')


In [52]:
df_toronto['Latitude']=0 #Creating new columns latitude and longitude and setting the default value to be one
df_toronto['Longitude']=0
list_lat=[]
list_lon=[]
for index, row in df_toronto.iterrows():
    list_lat.append(df_lat_ind.loc[row['Postal Code']][0])   #Iterating through the rows to get the corresponding Latitude and Longitude values for the postal codes available and storing them in a list
    list_lon.append(df_lat_ind.loc[row['Postal Code']][1])
df_toronto['Latitude']=list_lat
df_toronto['Longitude']=list_lon  #assigning the lists to the columns

In [59]:
df_toronto.head(10) #Given here is the final Dataset that is to be used for the clustering 

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2R,North York,Willowdale West,43.782736,-79.442259
4,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
5,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
6,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
8,M4P,Central Toronto,Davisville North,43.712751,-79.390197
13,M9P,Etobicoke,Westmount,43.696319,-79.532242
14,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
15,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
16,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817
19,M1J,Scarborough,Scarborough Village,43.744734,-79.239476


Latitude     43.706748
Longitude   -79.594054
Name: M9W, dtype: float64